In [1]:
import pandas as pd
import numpy as np

from keras import models, layers
from keras.utils import to_categorical, normalize
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
IDS2017_data = pd.read_csv("FlowDataSet/AppDDoS.csv", header = 0)
IDS2017_data = IDS2017_data.dropna()

print("IDS 2017 Data Shape: " + str(IDS2017_data.shape))

IDS 2017 Data Shape: (132394, 49)


In [3]:
IDS2017_data = IDS2017_data.dropna()
IDS2017_data['class'].replace({'normal' : 'NORMAL'}, inplace=True)
IDS2017_data['class'].replace({'slowbody2' : 'ATTACK', 'slowheaders' : 'ATTACK', 'slowloris' : 'ATTACK'}, inplace=True)
IDS2017_data['class'].replace({'ddossim' : 'ATTACK', 'hulk' : 'ATTACK', 'rudy' : 'ATTACK'}, inplace=True)
IDS2017_data['class'].replace({'slowread' : 'ATTACK', 'goldeneye' : 'ATTACK'}, inplace=True)
# IDS2017_data.replace({'class' : 'Class'})
IDS2017_data['class'].replace({'NORMAL' : 0, 'ATTACK' : 1}, inplace=True)
print(IDS2017_data['class'].value_counts())
print()
IDS2017_data.groupby('class').mean()

0    113438
1     18956
Name: class, dtype: int64



,srcport,dstport,proto,total_fpackets,total_fvolume,total_bpackets,total_bvolume,min_fpktl,mean_fpktl,max_fpktl,...,fpsh_cnt,bpsh_cnt,furg_cnt,burg_cnt,total_fhlen,total_bhlen,dscp,firstTime,flast,blast
class,,,,,,,,,,,,,,,,,,,,,
0,16914.530775,625.602620,9.094289,17.700286,2210.427026,26.149394,27997.918378,50.355630,69.857949,141.395873,...,2.997426,4.813034,0.0,0.0,743.469419,1202.785592,0.159735,1.276350e+09,1.276352e+09,1.276352e+09
1,33252.077231,80.689386,6.000000,26.153619,1833.116586,40.731378,50491.615214,45.592952,96.536928,273.891802,...,3.218981,2.048692,0.0,0.0,1133.527959,1692.528593,0.341633,1.276355e+09,1.276356e+09,1.276356e+09


In [4]:
ignore_features = [
    'srcip',
    'srcport',
    'dstip',
    'dstport',
    'proto',
    'total_fpackets',
    'total_fvolume',
    'total_bpackets',
    'total_bvolume',
    # 'min_fpktl',
    # 'mean_fpktl',
    # 'max_fpktl',
    'std_fpktl',
    # 'min_bpktl',
    # 'mean_bpktl',
    # 'max_bpktl',
    'std_bpktl',
    # 'min_fiat',
    # 'mean_fiat',
    # 'max_fiat',
    'std_fiat',
    # 'min_biat',
    # 'mean_biat',
    # 'max_biat',
    'std_biat',
    # 'duration',
    # 'min_active',
    # 'mean_active',
    # 'max_active',
    'std_active',
    'min_idle',
    'mean_idle',
    'max_idle',
    'std_idle',
    'sflow_fpackets',
    'sflow_fbytes',
    'sflow_bpackets',
    'sflow_bbytes',
    'fpsh_cnt',
    # 'bpsh_cnt',
    'furg_cnt',
    'burg_cnt',
    'total_fhlen',
    'total_bhlen',
    'dscp',
    'firstTime',
    'flast',
    'blast',
    'class'
]

In [5]:
data = IDS2017_data
y_ = data['class']
# x_ = data.drop(['class', 'srcip', 'dstip', 'srcport', 'dstport'], axis=1)
x_ = data.drop(columns=ignore_features, axis=1)
x_ = x_.astype(float)
x_ = normalize(x_)

y_ = y_.to_numpy()
x_ = x_.to_numpy()

y_ = to_categorical(y_)

In [6]:
X_train, X_test, y_train, y_test = train_test_split(x_, y_, test_size = 0.125)
print(X_train.shape)
print(X_test.shape)

# Transforming into One-Hot Encoding
lb = LabelBinarizer()
y_train = lb.fit_transform(y_train)
y_test = lb.transform(y_test)

print(y_train.shape)
print(y_test.shape)

(115844, 17)
(16550, 17)
(115844, 2)
(16550, 2)


In [7]:
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

print(X_train.shape)
print(X_test.shape)

(115844, 17, 1)
(16550, 17, 1)


In [8]:
# TEST_1
"""model = models.Sequential()

model.add(layers.Conv1D(32, 3, kernel_initializer='he_normal', activation='relu', padding='same', input_shape=(44, 1)))
model.add(layers.BatchNormalization())
model.add(layers.Conv1D(32, 3, kernel_initializer='he_normal', activation='relu', padding='same'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling1D(pool_size = 2, strides = 1, padding='same'))
model.add(layers.Dropout(0.1))

model.add(layers.Conv1D(64, 3, activation='elu', padding='same'))
model.add(layers.BatchNormalization())
model.add(layers.Conv1D(64, 3, activation='elu', padding='same'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling1D(pool_size = 2, strides = 1, padding='same'))
model.add(layers.Dropout(0.1))

model.add(layers.Conv1D(128, 3, activation='elu', padding='same'))
model.add(layers.BatchNormalization())
model.add(layers.Conv1D(128, 3, activation='elu', padding='same'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling1D(pool_size = 2, strides = 1, padding='same'))
model.add(layers.Dropout(0.1))

model.add(layers.Flatten())
model.add(layers.Dense(64, activation='elu', kernel_initializer='he_normal'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.4))
model.add(layers.Dense(9, kernel_initializer='he_normal', activation='softmax'))

print(model.summary())"""

# TEST_2
model = models.Sequential()

model.add(layers.Conv1D(32, 3, activation='relu', padding="same", input_shape=(17,1)))
model.add(layers.Conv1D(32, 3, padding="same", activation='relu'))
model.add(layers.MaxPooling1D(pool_size = 2, strides = 1, padding = 'same'))

model.add(layers.Conv1D(64, 3, activation = 'relu', padding = 'same'))
model.add(layers.Conv1D(64, 3, activation = 'relu', padding = 'same'))
model.add(layers.MaxPooling1D(pool_size = 2, strides = 1, padding = 'same'))

model.add(layers.Conv1D(96, 3, activation = 'relu', padding = 'same'))
model.add(layers.Conv1D(96, 3, activation = 'relu', padding = 'same'))
model.add(layers.MaxPooling1D(pool_size = 2, strides = 1, padding = 'same'))

model.add(layers.Flatten())
model.add(layers.Dense(90, activation = 'relu', input_shape = (10,)))

model.add(layers.Dropout(0.4))
model.add(layers.Dense(2, activation = 'softmax'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 17, 32)            128       
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 17, 32)            3104      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 17, 32)            0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 17, 64)            6208      
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 17, 64)            12352     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 17, 64)            0         
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 17, 96)           

In [9]:
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

reduce_lr = ReduceLROnPlateau(
    monitor='loss',
    factor=0.2,
    patience=5,
    verbose=0,
    min_delta=0.0001
)

checkpoint = ModelCheckpoint(
    filepath='IDS2017-RT3.h5',
    monitor='accuracy',
    mode='max',
    save_best_only=True,
    verbose=0
)

callbacks = [checkpoint, reduce_lr]

model.compile(
    loss='categorical_crossentropy',
    optimizer = Adam(lr = 0.001),
    metrics=['accuracy'])

model.fit(X_train, y_train, epochs = 5, batch_size = 128, verbose = 1, callbacks = callbacks)
test_loss, test_acc = model.evaluate(X_test, y_test)
print(test_acc)

Epoch 1/5
115844/115844 [==============================] - 66s 566us/step - loss: 0.2887 - accuracy: 0.8924
Epoch 2/5
115844/115844 [==============================] - 87s 748us/step - loss: 0.2433 - accuracy: 0.9068
Epoch 3/5
115844/115844 [==============================] - 86s 740us/step - loss: 0.2323 - accuracy: 0.9077
Epoch 4/5
115844/115844 [==============================] - 87s 754us/step - loss: 0.2252 - accuracy: 0.9083
Epoch 5/5
16550/16550 [==============================] - 6s 346us/step
0.9139577150344849


In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print(test_acc)